### Imports etc

In [1]:
import sys
from pathlib import Path
import pandas as pd
import seaborn as sns

REPO_PATH = Path.home() / "workspace/jetpointnet"
SCRIPT_PATH = REPO_PATH / "python_scripts"
sys.path.append(str(SCRIPT_PATH))

import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ''
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# from mpl_toolkits.axes_grid1 import make_axes_locatable
from jets_training.models.JetPointNet import PointNetSegmentation
from jets_training.jets_visualize import generate_images_and_metadata
from jets_training.jets_train import (
    TRAIN_INPUTS,
    MAX_SAMPLE_LENGTH,
    baseline_configuration,
    EXPERIMENT_NAME,
    TRAIN_TARGETS
)
OUTPUT_ACTIVATION_FUNCTION = baseline_configuration['OUTPUT_ACTIVATION_FUNCTION']




2024-07-30 00:05:38.899020: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-30 00:05:38.952255: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 00:05:38.952293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 00:05:38.953677: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 00:05:38.963737: I tensorflow/core/platform/cpu_feature_guar

Logged in as jhimmens


# Generate Event Images

In [2]:
model_path = "/home/jhimmens/workspace/jetpointnet/models/rev_6/collected_data/PointNet_epoch=10_name=distinctive-star-975.keras"
sets_to_visualize = ["rho"]
max_events_per_set = 10_000
max_images_per_set = 20

MODEL_NAME = model_path.split('/')[-1]
VISUAL_PATH = REPO_PATH / "visualizations" / EXPERIMENT_NAME / MODEL_NAME

model = PointNetSegmentation(MAX_SAMPLE_LENGTH, 
                             num_features=len(TRAIN_INPUTS), 
                             num_classes=len(TRAIN_TARGETS), 
                             output_activation_function=OUTPUT_ACTIVATION_FUNCTION,
                             model_version=1)

model.load_weights(model_path)

generate_images_and_metadata(sets_to_visualize, model, max_events_per_set, max_images_per_set, VISUAL_PATH)

2024-07-30 00:05:42.111045: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-30 00:05:42.111083: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: triumf-ml2
2024-07-30 00:05:42.111088: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: triumf-ml2
2024-07-30 00:05:42.111225: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: INVALID_ARGUMENT: expected %d.%d, %d.%d.%d, or %d.%d.%d.%d form for driver version; got "1"
2024-07-30 00:05:42.111249: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 550.90.7


  0%|          | 0/1 [00:00<?, ?it/s]

rho
/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_6/SavedNpz/deltaR=0.2_maxLen=800_MaxTrackAtributions=26_EScale=1/val


rho: 0it [00:00, ?it/s]

/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_6/SavedNpz/deltaR=0.2_maxLen=800_MaxTrackAtributions=26_EScale=1/val/rho/user.mswiatlo.40097513._000008.mltree.root_chunk_26_val.parquet.npz
5/5 [==============================] - 2s 103ms/step
saving: event=000927048_track=000
saving: event=000927032_track=000
saving: event=000927078_track=000
saving: event=000927076_track=000
saving: event=000927083_track=000
saving: event=000927095_track=000
saving: event=000927082_track=000
saving: event=000927037_track=000
saving: event=000927081_track=000
saving: event=000927112_track=000
saving: event=000927121_track=000
saving: event=000927098_track=000
saving: event=000927125_track=000
saving: event=000927130_track=000
saving: event=000927143_track=000
saving: event=000927145_track=000
saving: event=000927141_track=000
saving: event=000927158_track=000
saving: event=000927161_track=000
saving: event=000927149_track=000
/fast_scratch_1/atlas/pflow/jh